In [1]:
import pandas as pd
import argparse
from openai import OpenAI, AsyncOpenAI
import os
from tqdm import tqdm
import emoji
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from inference import llama3_inference
import re
from calc_scores import calc_and_save_acc
from utils import crop_predictions

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader 
from tqdm import tqdm
import emoji
import argparse
from  prompts import PROMPTS

In [8]:

def eval_llama(prompt, clue, target, model, tokenizer, ans='',  definition = ' '):
    correct = 0
    
    messages = [
    {"role": "user", "content": prompt.format(clue=clue, ans=ans, definition=definition)},
]
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)
    
    response = llama3_inference (model, tokenizer, [prompt],do_sample= False,temp=0.1, top_p=0.1 )[0].lower()
    # response = model.generate([prompt], sampling_params, use_tqdm= False)[0].outputs[0].text.strip().lower()

    # print(response)

    for l in response.split('\n'):
            if 'answer:' in l:
                response = l.split('answer:')[1].strip().replace(',','').replace('.','').replace('?','').replace('!','').replace('*','').strip()

    if re.findall('"([^"]*)"', response):
        response = re.findall('"([^"]*)"', response)[0]
    
    elif 'definition word' in response:
        response = response.split('definition word is:')[1]
    elif 'wordplay type is' in response:
        response = response.split('wordplay type is:')[1]
    


    # response = response.split('\n')[0]
    for d in target:
        if str(d).strip() == response.strip():
            correct = 1
            break
        else:
            continue

    return correct, response

In [3]:
# model = AutoModelForCausalLM.from_pretrained(
#     'google/gemma-2-9b-it',
#     attn_implementation= "flash_attention_2",
#     torch_dtype = torch.bfloat16,
#     device_map = 'auto'
# )
# model = model.eval()
# tokenizer = AutoTokenizer.from_pretrained('google/gemma-2-9b-it')
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id
# model.config.pad_token_id = model.config.bos_token_id

import os
import sys
os.environ["VLLM_ATTENTION_BACKEND"] = "FLASHINFER"

from vllm import LLM, SamplingParams
model = LLM(
    model="google/gemma-2-9b-it",
    # gpu_memory_utilization=0.9,
    max_model_len=256
)




WARNING 07-28 08:30:53 utils.py:569] Gemma 2 uses sliding window attention for every odd layer, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 07-28 08:30:53 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=256, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-28 08:31:13 model_runner.py:692] Loading model weights took 17.3781 GB
INFO 07-28 08:31:14 gpu_executor.py:102] # GPU blocks: 2990, # CPU blocks: 780
INFO 07-28 08:31:18 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-28 08:31:18 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-28 08:31:38 model_runner.py:1181] Graph capturing finished in 19 secs.


In [4]:
tokenizer = model.get_tokenizer()

sampling_params = SamplingParams(
    temperature=0.0, top_p=1, max_tokens=256,
    stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
)


In [5]:
dataset = pd.read_csv("data/georgo_ho_clues_sampled.csv")

dataset = dataset.sample(frac=0.015).reset_index(drop=True)


In [9]:
p = """You are a cryptic crossword expert. I will give you a cryptic clue. Every clue has two parts: a definition and a wordplay. The definition is a synonym of the clue's answer. Your task is to identify and extract the word/s in the clue. Only output the definition.
Clue: {clue}
Definition:
"""

for i, row in tqdm(dataset.iterrows(),total=dataset.shape[0]):

    definition_label = str(row['Definition']).lower()
    if '/' in definition_label:
        definition_label = definition_label.split('/')
    else:
        definition_label = [definition_label]

    for d in definition_label:
        d = d.strip()
    clue = str(row['Clue'])
    ans = str(row['Answer'])
    # correct, response = eval_llama(p, clue, definition_label, model, tokenizer)
    correct, response = eval_llama(PROMPTS['DEFINITION_PROMPT'], clue, definition_label, model, tokenizer)
    

    print(clue)
    print(definition_label)
    print(response)

  0%|                                                                                                                                                                                       | 0/15 [00:00<?, ?it/s]


AttributeError: 'LLM' object has no attribute 'device'

In [ ]:
with open(args.output_file, 'w') as f:
    f.write(f'Evaluation of {args.model}\n\n')

    definition = PROMPTS[args.prompt]
    wordplay = PROMPTS[args.prompt]
    f.write(f'Prompts: \n Definition_prompt: {definition} \n Wordplay_prompt:{wordplay  }\n\n')
    f.write(f'Definition Accuracy: {definition_acc/len(dataset)}\n')
    f.write(f'Wordplay Accuracy: {wordplay_acc/len(dataset)}\n')
    f.write('\n\n')
    f.write('Definition Responses\n\n')
    for res in definition_responses:
        f.write(res)
    f.write('\n\n')
    f.write('Wordplay Responses\n\n')
    for res in wordplay_responses:
        f.write(res)
    f.write('\n\n') 
    f.write(f'Total Clues: {len(dataset)}')
